In [11]:
#pip install spark-nlp NOT USED
#pip install pyspark NOT USED


#pip install --upgrade pymupdf
#pip install -U sentence-transformers

import pymupdf #for reading pdf files
from sentence_transformers import SentenceTransformer #for getting vectors from text
from pathlib import Path #for reading all pdfs in the folder
import pandas as pd
import spacy
from collections import Counter
from string import punctuation

nlp = spacy.load("en_core_web_sm")
restaurantdf = pd.DataFrame(columns=["Restaurant Name", "Keywords", "Embeddings"])

def pdf_reader(file_path): #test method for reading pdfs
    #open file
    file = pymupdf.open(file_path)
    
    #read the file
    #out = open("output.txt", "wb") #prints text to output file in the directory
    out = ""
    for page in file: #iterate the pages
        text = page.get_text() #get plain text
        out += text #write text of page
    #print(out)
        
    file.close()
    return out

def txt_reader(): #test method for reading text files
    #open file
    file = open("testerfile.txt")
    
    #read the file
    content = file.read()
    print(content)
    
    #close the file
    file.close()
    return

In [12]:
def main():
    #doc = pymupdf.open("")
    #txt_reader()
    #print(pdf_reader("Merged March 2023 (wine).pdf"))
    return

if __name__ == "__main__":
    main()

In [13]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

sentence1 = "The weather is lovely today."
sentence2 = "It's so sunny outside!"
sentence3 = "He drove to the stadium."

# The sentences to encode
sentences = [
    sentence1,
    sentence2,
    sentence3,
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]
#embedding1 = embeddings[0] #print the embedding for sentence1 in the index 0
#print(embedding1)

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [14]:
#logic for ingesting the menus and adding them to a vector space

# returns all file paths that has .pdf as extension in the specified directory, from https://stackoverflow.com/questions/70848337/how-to-read-multiple-pdf-from-a-folder-one-by-one
pdf_search = Path(str(Path.cwd())).rglob("*.pdf") #gets all the pdf files in the current directory
pdf_files = [str(file.absolute()) for file in pdf_search] #iterates over all the pdf files in the current directory and adds them to an array.

pdf_filecontent = [pdf_reader(file) for file in pdf_files] #take the file names and extract the text content from them.

menuembeddings = model.encode(pdf_filecontent)
print(menuembeddings.shape)
menuembedding1 = menuembeddings[2] #using the wine restuarant to tests similarity scores


prompt0 = "I like wine and different alcoholic drinks like pinot and red wine, cabernet sauvignon, rose wine, and sauvignon blanc, any wine or beer is good."
prompt1 = "I want to eat cheeseburgers and sandwiches, they're some of my favorite foods to eat."
prompt2 = "The Dodgers are the best baseball team to ever exist, I like to watch their games on sundays."
prompts = [prompt0, prompt1, prompt2]
promptembeddings = model.encode(prompts)

similarities = []
for i, embedding in enumerate(promptembeddings):
    similarity = model.similarity(embedding, menuembedding1) #compare each prompt in the prompt array to the wine restuarant.
    similarities.append(similarity.item()) #append the similarity score to the array in a scalar format
    
print(similarities)


(4, 384)
[0.33162087202072144, 0.37901878356933594, 0.07464508712291718]


In [15]:

# returns all file paths that has .pdf as extension in the specified directory, from https://stackoverflow.com/questions/70848337/how-to-read-multiple-pdf-from-a-folder-one-by-one
pdf_search = Path(str(Path.cwd())).rglob("*.pdf") #gets all the pdf files in the current directory
pdf_files = [str(file.absolute()) for file in pdf_search] #iterates over all the pdf files in the current directory and adds them to an array.

pdf_filecontent = [pdf_reader(file) for file in pdf_files] #take the file names and extract the text content from them.

menuembeddings = model.encode(pdf_filecontent)
print(menuembeddings.shape)
menuembedding1 = menuembeddings[2] #using the wine restuarant to tests similarity scores


prompt0 = "I like wine and different alcoholic drinks like pinot and red wine, cabernet sauvignon, rose wine, and sauvignon blanc, any wine or beer is good."
prompt1 = "I want to eat cheeseburgers and sandwiches, they're some of my favorite foods to eat."
prompt2 = "the Dodgers are the best baseball team to ever exist, I like to watch their games on sundays."
prompts = [prompt0, prompt1, prompt2]
promptembeddings = model.encode(prompts)

similarities = []
for i, embedding in enumerate(promptembeddings):
    similarity = model.similarity(embedding, menuembedding1) #compare each prompt in the prompt array to the wine restuarant.
    similarities.append(similarity.item()) #append the similarity score to the array in a scalar format
    
print(similarities)


(4, 384)
[0.33162087202072144, 0.37901878356933594, 0.07464508712291718]


In [16]:
#echo bot code

"""st.title("Restuarant Query Bot") #page title
    with st.chat_message("assistant"): #gives name to bot and writes welcome message
        st.write("Hello, I'm your restuarant assistant, how can I help you?")
        
    if "messages" not in st.session_state: #if no messages have been sent then initialize chat history
        st.session_state.messages = [] #variable for chat history
        
    for message in st.session_state.messages: #loop that iterates through the chat history to display the history in the message container.
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    if prompt := st.chat_input("Say something"):
        st.chat_message("user").markdown(prompt) #display message in chat message container
        st.session_state.messages.append({"role":"user", "content": prompt}) #add message to chat history
        
        response = f"{prompt}"
        with st.chat_message("assistant"):
            st.markdown(response)
            st.session_state.messages.append({"role": "assistant", "content": response})
    return"""

'st.title("Restuarant Query Bot") #page title\n    with st.chat_message("assistant"): #gives name to bot and writes welcome message\n        st.write("Hello, I\'m your restuarant assistant, how can I help you?")\n        \n    if "messages" not in st.session_state: #if no messages have been sent then initialize chat history\n        st.session_state.messages = [] #variable for chat history\n        \n    for message in st.session_state.messages: #loop that iterates through the chat history to display the history in the message container.\n        with st.chat_message(message["role"]):\n            st.markdown(message["content"])\n    \n    if prompt := st.chat_input("Say something"):\n        st.chat_message("user").markdown(prompt) #display message in chat message container\n        st.session_state.messages.append({"role":"user", "content": prompt}) #add message to chat history\n        \n        response = f"{prompt}"\n        with st.chat_message("assistant"):\n            st.markd

In [23]:
restuarantdf = pd.DataFrame(["Restuarant Name", "Keywords", "Embeddings"])

def restuarant_datapull(text): #uses spaCy to extract restuarant keywords, and add the title, keywords, and embeddings to a data frame.
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] #three labels for the types of extracted words
    doc = nlp(text.lower()) #lowercase all words
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation): #if word is in the stop words or is punctuation move on
            continue
        if(token.pos_ in pos_tag): #if not add to keywords
            result.append(token.text)
    return result

new_text = """
When it comes to evaluating the performance of keyword extractors, you can use some of the standard metrics in machine learning: accuracy, precision, recall, and F1 score. However, these metrics don’t reflect partial matches. they only consider the perfect match between an extracted segment and the correct prediction for that tag.
Fortunately, there are some other metrics capable of capturing partial matches. An example of this is ROUGE.
"""
output = set(restuarant_datapull(new_text))
most_common = Counter(output).most_common(10)
for item in most_common:
    print(item[0])

keyword
performance
prediction
standard
rouge
learning
partial
perfect
accuracy
machine


In [ ]:
def get_restuarant_info():
    restaurantdf
    restaurant_titles = get_restaurantnames()
    restaurant_texts = get_restauranttext()
    
    for name, text in zip(restaurant_titles, restaurant_texts):
        keywords = restaurant_datapull(text)
        embeddings = get_embeddings(keywords)
        restaurantdf.loc[len(restaurantdf)] = [name, keywords, embeddings]
    return restaurantdf